# Employee Attrition – Preprocessing & Train/Validation Split

## Objective
Prepare a clean, reproducible preprocessing pipeline for modeling:
- define target and feature set
- drop non-informative / identifier columns
- split train/validation with stratification
- build a ColumnTransformer pipeline (encoding + scaling)

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.compose import ColumnTransformer

In [5]:
from sklearn.pipeline import Pipeline

In [6]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [7]:
from sklearn.impute import SimpleImputer

In [8]:
import joblib

In [9]:
DATA_PATH = "../data/raw/WA_Fn-UseC_-HR-Employee-Attrition.csv"

In [10]:
df = pd.read_csv(DATA_PATH)

In [11]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [12]:
TARGET_COL = 'Attrition'

In [13]:
y = df[TARGET_COL].map({'No':0, 'Yes':1}).astype(int)

In [14]:
constant_cols = [col for col in df.columns if df[col].nunique() == 1]
constant_cols

['EmployeeCount', 'Over18', 'StandardHours']

In [15]:
drop_cols = [
    TARGET_COL,
    'EmployeeCount',
    'Over18',
    'StandardHours',
    'EmployeeNumber' #id
]

In [16]:
X = df.drop(columns = drop_cols)

In [17]:
X.shape, y.value_counts(normalize = True), df.shape

((1470, 30),
 Attrition
 0    0.838776
 1    0.161224
 Name: proportion, dtype: float64,
 (1470, 35))

In [18]:
assert set(y.unique()) == {0, 1}

In [19]:
assert y.nunique() == 2

In [20]:
for col in ['EmployeeCount', 'StandardHours', 'Over18']:
    if col in df.columns:
        print(col, df[col].nunique(),df[col].unique()[:5])

EmployeeCount 1 [1]
StandardHours 1 [80]
Over18 1 ['Y']


In [21]:
for c in drop_cols:
    assert c not in X.columns, f"{c} still in features!"

In [22]:
X_train, X_valid, y_train, y_valid = train_test_split (
    X,y,
    test_size = 0.2,
    random_state = 42,
    stratify = y
)

In [23]:
X_train.shape, X_valid.shape, y_train.mean(), y_valid.mean()

((1176, 30),
 (294, 30),
 np.float64(0.16156462585034015),
 np.float64(0.1598639455782313))

In [24]:
numeric_features = X_train.select_dtypes(include = ['int64', 'float64']).columns.tolist()

In [25]:
categorical_features = X_train.select_dtypes(include = ['object']).columns.tolist()

In [26]:
len(numeric_features), len(categorical_features)

(23, 7)

In [27]:
numeric_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scaler', StandardScaler())
])

In [28]:
categorical_transformer = Pipeline(steps =[
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown = 'ignore', sparse_output = False))
])

In [29]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_transformer,numeric_features),
        ('cat', categorical_transformer,categorical_features)
    ],
    remainder = 'drop'
)

In [30]:
preprocessor

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e.g. `

In [31]:
X_train_prep = preprocessor.fit_transform(X_train)

In [32]:
X_valid_prep = preprocessor.transform(X_valid)

In [33]:
X_train_prep.shape, X_valid_prep.shape

((1176, 51), (294, 51))

In [34]:
ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']

In [35]:
cat_feature_names = ohe.get_feature_names_out(categorical_features)

In [36]:
feature_names = np.concatenate([numeric_features,cat_feature_names])

In [37]:
assert X_train_prep.shape[1] == len(feature_names)

In [38]:
len(feature_names), feature_names[:20]

(51,
 array(['Age', 'DailyRate', 'DistanceFromHome', 'Education',
        'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement',
        'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate',
        'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating',
        'RelationshipSatisfaction', 'StockOptionLevel',
        'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
        'YearsAtCompany'], dtype=object))

In [39]:
joblib.dump(preprocessor, '../model/preprocessor.joblib')

['../model/preprocessor.joblib']